# Simple Agent Movement Demo

## Educational Overview

This notebook demonstrates the fundamental concepts of odor plume navigation by showcasing a simple agent moving through a Gaussian odor field environment. You'll learn:

- **Navigation Concepts**: How agents navigate using position, orientation, and speed parameters
- **Odor Field Generation**: Creating realistic odor environments with multiple Gaussian sources
- **Real-time Visualization**: Animating agent movement with interactive parameter control
- **Reproducible Research**: Using seed management for consistent experimental results
- **Configuration Management**: Dynamic parameter assembly using Hydra Compose API

### Key Learning Objectives

1. **Understand Agent State**: Learn how agents are represented with position, orientation, speed, and angular velocity
2. **Environment Interaction**: See how agents sample odor concentrations from their environment
3. **Simulation Dynamics**: Observe how agent state evolves over time through discrete time steps
4. **Visualization Techniques**: Master real-time animation and static trajectory plotting
5. **Research Reproducibility**: Apply seed management for consistent experimental results

---

## 1. Environment Setup and Imports

First, let's import all necessary libraries and set up our environment. We'll use the new project structure with imports from the refactored library.

In [ ]:
# Core scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
import time
import warnings
from typing import Tuple, List, Optional, Dict, Any

# Hydra imports for dynamic configuration
try:
    from hydra import compose, initialize
    from omegaconf import DictConfig, OmegaConf
    HYDRA_AVAILABLE = True
    print("✓ Hydra configuration system available")
except ImportError:
    HYDRA_AVAILABLE = False
    print("⚠ Hydra not available, using fallback configuration")
    
# Interactive widgets for Jupyter
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print("✓ Interactive widgets available")
except ImportError:
    WIDGETS_AVAILABLE = False
    print("⚠ IPython widgets not available, interactive features disabled")

# Project imports using new structure
from {{cookiecutter.project_slug}}.core.navigator import NavigatorFactory, NavigatorProtocol
from {{cookiecutter.project_slug}}.utils.visualization import SimulationVisualization
from {{cookiecutter.project_slug}}.utils.seed_manager import set_global_seed, seed_context
from {{cookiecutter.project_slug}}.config.schemas import SingleAgentConfig

print("✓ All core imports successful")
print("✓ Ready to begin simple movement demonstration")

## 2. Seed Management for Reproducible Research

Before starting our simulations, let's establish reproducible random state management. This ensures that our experiments produce consistent results across different runs and environments.

In [ ]:
# Set up reproducible random state
EXPERIMENT_SEED = 42
EXPERIMENT_ID = "simple_movement_demo"

# Initialize global seed manager
seed_manager = set_global_seed(
    seed=EXPERIMENT_SEED, 
    experiment_id=EXPERIMENT_ID,
    enable_logging=True
)

print(f"🌱 Seed manager initialized with seed={EXPERIMENT_SEED}")
print(f"📋 Experiment ID: {EXPERIMENT_ID}")
print(f"⏱ Initialization time: {seed_manager._initialization_time:.2f}ms")

# Capture initial state for later restoration if needed
initial_state = seed_manager.capture_state()
print(f"📸 Initial random state captured (checksum: {initial_state.state_checksum})")

# Display reproducibility information
repro_info = seed_manager.get_reproducibility_info()
print("\n🔬 Reproducibility Information:")
print(f"  Platform: {repro_info['platform_info']['platform']}")
print(f"  Python: {'.'.join(map(str, repro_info['platform_info']['python_version']))}")
print(f"  NumPy: {repro_info['platform_info']['numpy_version']}")
print(f"  Architecture: {repro_info['platform_info']['architecture']}")

## 3. Configuration Management with Hydra

Let's set up dynamic configuration management using Hydra's Compose API. This allows us to assemble configurations programmatically and modify parameters interactively.

In [ ]:
def create_configuration(config_overrides: Optional[Dict[str, Any]] = None) -> DictConfig:
    """Create dynamic configuration using Hydra Compose API."""
    
    if HYDRA_AVAILABLE:
        try:
            # Initialize Hydra with relative path to conf directory
            with initialize(version_base=None, config_path="../../conf"):
                # Compose configuration with optional overrides
                cfg = compose(config_name="base", overrides=config_overrides or [])
                print("✓ Hydra configuration loaded successfully")
                return cfg
        except Exception as e:
            print(f"⚠ Hydra configuration failed: {e}")
            print("📝 Falling back to manual configuration")
    
    # Fallback configuration if Hydra is not available
    fallback_config = {
        'simulation': {
            'duration': 40,
            'timestep': 0.5,
            'environment': {
                'width': 50,
                'height': 50,
                'odor_sources': [
                    {'x': 35.0, 'y': 30.0, 'intensity': 1.0, 'sigma': 5.0},
                    {'x': 15.0, 'y': 15.0, 'intensity': 0.7, 'sigma': 7.0}
                ]
            }
        },
        'navigator': {
            'position': [5.0, 5.0],
            'orientation': 45.0,
            'speed': 0.5,
            'max_speed': 1.0,
            'angular_velocity': 0.0
        },
        'visualization': {
            'figsize': [10, 8],
            'dpi': 150,
            'fps': 30,
            'theme': 'scientific'
        }
    }
    
    # Apply overrides to fallback config
    if config_overrides:
        for override in config_overrides:
            if '=' in override:
                key_path, value = override.split('=', 1)
                # Simple override implementation for fallback
                keys = key_path.split('.')
                config_section = fallback_config
                for key in keys[:-1]:
                    config_section = config_section.setdefault(key, {})
                try:
                    # Try to convert value to appropriate type
                    if value.lower() in ['true', 'false']:
                        config_section[keys[-1]] = value.lower() == 'true'
                    elif '.' in value:
                        config_section[keys[-1]] = float(value)
                    else:
                        config_section[keys[-1]] = int(value)
                except ValueError:
                    config_section[keys[-1]] = value
    
    return OmegaConf.create(fallback_config) if HYDRA_AVAILABLE else fallback_config

# Create initial configuration
config = create_configuration()
print("\n⚙️ Initial Configuration:")
if HYDRA_AVAILABLE:
    print(OmegaConf.to_yaml(config))
else:
    import json
    print(json.dumps(config, indent=2))

## 4. Odor Field Generation

Now let's create a realistic odor environment using Gaussian distributions. This simulates multiple odor sources with different intensities and spread patterns.

In [ ]:
def create_gaussian_odor_field(width: int, height: int, odor_sources: List[Dict[str, float]]) -> np.ndarray:
    """
    Create a realistic Gaussian odor field with multiple sources.
    
    This function generates a 2D odor concentration field by combining multiple
    Gaussian distributions representing different odor sources. Each source is
    characterized by its position (x, y), intensity, and spread (sigma).
    
    Args:
        width: Environment width in grid units
        height: Environment height in grid units  
        odor_sources: List of source dictionaries with keys: x, y, intensity, sigma
    
    Returns:
        2D numpy array with normalized odor concentrations [0, 1]
    """
    print(f"🌬️ Generating odor field ({width}×{height}) with {len(odor_sources)} sources")
    
    # Create coordinate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    odor_field = np.zeros((height, width), dtype=np.float32)
    
    # Add each Gaussian odor source
    for i, source in enumerate(odor_sources):
        cx, cy = source['x'], source['y']
        intensity = source['intensity']
        sigma = source['sigma']
        
        # Calculate Gaussian distribution
        gaussian = np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
        contribution = intensity * gaussian
        odor_field += contribution
        
        print(f"  Source {i+1}: position=({cx:.1f}, {cy:.1f}), intensity={intensity:.2f}, sigma={sigma:.1f}")
        print(f"    Peak contribution: {np.max(contribution):.3f}")
    
    # Normalize field to [0, 1] range
    max_concentration = np.max(odor_field)
    if max_concentration > 0:
        odor_field = odor_field / max_concentration
        print(f"  Field normalized by factor: {max_concentration:.3f}")
    
    print(f"  Final field range: [{np.min(odor_field):.3f}, {np.max(odor_field):.3f}]")
    return odor_field

# Extract environment configuration
env_config = config['simulation']['environment']
width = env_config['width']
height = env_config['height']
odor_sources = env_config['odor_sources']

# Generate the odor field
odor_field = create_gaussian_odor_field(width, height, odor_sources)

# Display basic statistics
print(f"\n📊 Odor Field Statistics:")
print(f"  Shape: {odor_field.shape}")
print(f"  Data type: {odor_field.dtype}")
print(f"  Mean concentration: {np.mean(odor_field):.4f}")
print(f"  Standard deviation: {np.std(odor_field):.4f}")
print(f"  Non-zero pixels: {np.count_nonzero(odor_field)} ({np.count_nonzero(odor_field)/(width*height)*100:.1f}%)")

### 4.1 Visualize the Odor Field

Let's visualize the generated odor field to understand the environment our agent will navigate through.

In [ ]:
# Create static visualization of the odor field
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Main odor field visualization
im1 = ax1.imshow(odor_field, origin='lower', cmap='viridis', extent=[0, width, 0, height])
ax1.set_title('Odor Concentration Field', fontsize=14, fontweight='bold')
ax1.set_xlabel('X Position')
ax1.set_ylabel('Y Position')
cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
cbar1.set_label('Concentration')

# Mark odor sources
for i, source in enumerate(odor_sources):
    ax1.scatter(source['x'], source['y'], c='red', s=100, marker='X', 
               edgecolors='white', linewidth=2, alpha=0.9,
               label=f'Source {i+1}' if i == 0 else '')
    ax1.annotate(f'S{i+1}\nσ={source["sigma"]:.1f}', 
                xy=(source['x'], source['y']), 
                xytext=(source['x']+2, source['y']+2),
                fontsize=9, color='white',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.7))

ax1.legend()
ax1.grid(True, alpha=0.3)

# Cross-section visualization
mid_y = height // 2
cross_section = odor_field[mid_y, :]
ax2.plot(range(width), cross_section, 'b-', linewidth=2, label=f'Y={mid_y} cross-section')
ax2.fill_between(range(width), cross_section, alpha=0.3)
ax2.set_title('Odor Concentration Cross-Section', fontsize=14, fontweight='bold')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Concentration')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Mark source positions in cross-section
for i, source in enumerate(odor_sources):
    if abs(source['y'] - mid_y) <= 2:  # Show sources near the cross-section
        concentration_at_source = odor_field[int(source['y']), int(source['x'])]
        ax2.axvline(source['x'], color='red', linestyle='--', alpha=0.7, 
                   label=f'Source {i+1}' if i == 0 else '')

plt.tight_layout()
plt.show()

print("✓ Odor field visualization complete")

## 5. Agent Configuration and Initialization

Now let's create and configure our navigation agent. We'll use the NavigatorFactory with our configuration to create a single agent.

In [ ]:
def create_navigator_from_config(config: Dict[str, Any]) -> NavigatorProtocol:
    """
    Create a navigator instance from configuration.
    
    This function demonstrates how to use the NavigatorFactory to create
    agents from configuration data, showcasing the configuration-driven
    approach to agent initialization.
    """
    nav_config = config['navigator']
    
    print(f"🤖 Creating navigator with configuration:")
    print(f"  Initial position: {nav_config['position']}")
    print(f"  Initial orientation: {nav_config['orientation']}°")
    print(f"  Initial speed: {nav_config['speed']} units/step")
    print(f"  Maximum speed: {nav_config['max_speed']} units/step")
    print(f"  Angular velocity: {nav_config['angular_velocity']}°/step")
    
    # Create single agent navigator
    navigator = NavigatorFactory.single_agent(
        position=tuple(nav_config['position']),
        orientation=nav_config['orientation'],
        speed=nav_config['speed'],
        max_speed=nav_config['max_speed'],
        angular_velocity=nav_config['angular_velocity']
    )
    
    print(f"✓ Navigator created successfully")
    return navigator

# Create the navigator
navigator = create_navigator_from_config(config)

# Display initial agent state
print(f"\n📍 Initial Agent State:")
print(f"  Number of agents: {navigator.num_agents}")
print(f"  Positions: {navigator.positions}")
print(f"  Orientations: {navigator.orientations}°")
print(f"  Speeds: {navigator.speeds} units/step")
print(f"  Max speeds: {navigator.max_speeds} units/step")
print(f"  Angular velocities: {navigator.angular_velocities}°/step")

# Sample initial odor concentration
initial_odor = navigator.sample_odor(odor_field)
print(f"  Initial odor concentration: {initial_odor:.4f}")

## 6. Interactive Parameter Exploration

Let's create interactive widgets to explore how different parameters affect agent behavior. This allows real-time parameter adjustment and immediate visualization of the effects.

In [ ]:
if WIDGETS_AVAILABLE:
    # Create interactive parameter controls
    print("🎛️ Creating interactive parameter controls...")
    
    # Agent parameter widgets
    position_x_widget = widgets.FloatSlider(
        value=config['navigator']['position'][0],
        min=0, max=width-1, step=0.5,
        description='Start X:',
        style={'description_width': 'initial'}
    )
    
    position_y_widget = widgets.FloatSlider(
        value=config['navigator']['position'][1],
        min=0, max=height-1, step=0.5,
        description='Start Y:',
        style={'description_width': 'initial'}
    )
    
    orientation_widget = widgets.FloatSlider(
        value=config['navigator']['orientation'],
        min=0, max=360, step=5,
        description='Orientation (°):',
        style={'description_width': 'initial'}
    )
    
    speed_widget = widgets.FloatSlider(
        value=config['navigator']['speed'],
        min=0, max=2.0, step=0.1,
        description='Speed:',
        style={'description_width': 'initial'}
    )
    
    max_speed_widget = widgets.FloatSlider(
        value=config['navigator']['max_speed'],
        min=0.1, max=3.0, step=0.1,
        description='Max Speed:',
        style={'description_width': 'initial'}
    )
    
    # Simulation parameter widgets
    duration_widget = widgets.IntSlider(
        value=config['simulation']['duration'],
        min=10, max=100, step=5,
        description='Duration (steps):',
        style={'description_width': 'initial'}
    )
    
    timestep_widget = widgets.FloatSlider(
        value=config['simulation']['timestep'],
        min=0.1, max=2.0, step=0.1,
        description='Time Step:',
        style={'description_width': 'initial'}
    )
    
    # Seed widget for reproducibility
    seed_widget = widgets.IntSlider(
        value=EXPERIMENT_SEED,
        min=1, max=1000, step=1,
        description='Random Seed:',
        style={'description_width': 'initial'}
    )
    
    # Control buttons
    run_button = widgets.Button(
        description='Run Simulation',
        button_style='success',
        icon='play'
    )
    
    reset_button = widgets.Button(
        description='Reset to Defaults',
        button_style='warning',
        icon='refresh'
    )
    
    # Output area for results
    output_area = widgets.Output()
    
    # Organize widgets in tabs
    agent_tab = widgets.VBox([
        widgets.HTML('<h3>Agent Parameters</h3>'),
        position_x_widget, position_y_widget, 
        orientation_widget, speed_widget, max_speed_widget
    ])
    
    simulation_tab = widgets.VBox([
        widgets.HTML('<h3>Simulation Parameters</h3>'),
        duration_widget, timestep_widget, seed_widget
    ])
    
    controls_tab = widgets.VBox([
        widgets.HTML('<h3>Controls</h3>'),
        widgets.HBox([run_button, reset_button])
    ])
    
    # Create tabbed interface
    tabs = widgets.Tab(children=[agent_tab, simulation_tab, controls_tab])
    tabs.set_title(0, 'Agent')
    tabs.set_title(1, 'Simulation')
    tabs.set_title(2, 'Controls')
    
    # Parameter update functions
    def update_config_from_widgets():
        """Update configuration from widget values."""
        config['navigator']['position'] = [position_x_widget.value, position_y_widget.value]
        config['navigator']['orientation'] = orientation_widget.value
        config['navigator']['speed'] = speed_widget.value
        config['navigator']['max_speed'] = max_speed_widget.value
        config['simulation']['duration'] = duration_widget.value
        config['simulation']['timestep'] = timestep_widget.value
        return config
    
    def reset_to_defaults(button):
        """Reset all widgets to default values."""
        with output_area:
            clear_output()
            print("🔄 Resetting parameters to defaults...")
            
        # Reset to original config values
        original_config = create_configuration()
        position_x_widget.value = original_config['navigator']['position'][0]
        position_y_widget.value = original_config['navigator']['position'][1]
        orientation_widget.value = original_config['navigator']['orientation']
        speed_widget.value = original_config['navigator']['speed']
        max_speed_widget.value = original_config['navigator']['max_speed']
        duration_widget.value = original_config['simulation']['duration']
        timestep_widget.value = original_config['simulation']['timestep']
        
        with output_area:
            print("✓ Parameters reset to defaults")
    
    reset_button.on_click(reset_to_defaults)
    
    # Display the interface
    display(widgets.VBox([tabs, output_area]))
    print("✓ Interactive parameter controls created")
    
else:
    print("⚠ Interactive widgets not available")
    print("📝 Using static parameter configuration")
    
    def update_config_from_widgets():
        """Fallback function when widgets are not available."""
        return config
    
    output_area = None

## 7. Simulation Runner with Step-by-Step Execution

Now let's implement the core simulation logic that moves our agent through the environment step by step, recording its trajectory and odor encounters.

In [ ]:
def run_simulation_step_by_step(navigator: NavigatorProtocol, 
                               environment: np.ndarray,
                               duration: int,
                               timestep: float,
                               verbose: bool = True) -> Dict[str, Any]:
    """
    Execute simulation step by step with detailed tracking.
    
    This function demonstrates the core simulation loop, showing how agents
    update their state over time through interaction with the environment.
    
    Args:
        navigator: Navigator instance to simulate
        environment: 2D environment array for odor sampling
        duration: Number of simulation steps
        timestep: Time step size for updates
        verbose: Whether to print progress information
        
    Returns:
        Dictionary containing complete simulation results
    """
    if verbose:
        print(f"🚀 Starting simulation: {duration} steps, dt={timestep}")
    
    # Initialize result storage
    results = {
        'positions': [],
        'orientations': [],
        'speeds': [],
        'odor_values': [],
        'timestamps': [],
        'step_details': []
    }
    
    # Record initial state
    initial_position = navigator.positions[0].copy()
    initial_orientation = navigator.orientations[0]
    initial_speed = navigator.speeds[0]
    initial_odor = navigator.sample_odor(environment)
    
    results['positions'].append(initial_position)
    results['orientations'].append(initial_orientation)
    results['speeds'].append(initial_speed)
    results['odor_values'].append(initial_odor)
    results['timestamps'].append(0.0)
    
    if verbose:
        print(f"📍 Initial state: pos={initial_position}, orient={initial_orientation:.1f}°, speed={initial_speed:.2f}")
        print(f"👃 Initial odor: {initial_odor:.4f}")
        print("\n⏯️ Simulation progress:")
    
    # Main simulation loop
    start_time = time.time()
    
    for step in range(duration):
        step_start = time.time()
        current_time = step * timestep
        
        # Execute one simulation step
        navigator.step(environment, timestep)
        
        # Record current state
        current_position = navigator.positions[0].copy()
        current_orientation = navigator.orientations[0]
        current_speed = navigator.speeds[0]
        current_odor = navigator.sample_odor(environment)
        
        results['positions'].append(current_position)
        results['orientations'].append(current_orientation)
        results['speeds'].append(current_speed)
        results['odor_values'].append(current_odor)
        results['timestamps'].append(current_time + timestep)
        
        # Calculate step statistics
        prev_position = results['positions'][-2]
        distance_moved = np.linalg.norm(current_position - prev_position)
        orientation_change = current_orientation - results['orientations'][-2]
        
        step_details = {
            'step': step + 1,
            'time': current_time + timestep,
            'position': current_position,
            'orientation': current_orientation,
            'speed': current_speed,
            'odor': current_odor,
            'distance_moved': distance_moved,
            'orientation_change': orientation_change,
            'execution_time': time.time() - step_start
        }
        results['step_details'].append(step_details)
        
        # Progress reporting
        if verbose and (step + 1) % max(1, duration // 10) == 0:
            progress = (step + 1) / duration * 100
            print(f"  Step {step+1:3d}/{duration}: {progress:5.1f}% | "
                  f"pos=({current_position[0]:5.1f}, {current_position[1]:5.1f}) | "
                  f"orient={current_orientation:6.1f}° | "
                  f"odor={current_odor:.4f}")
        
        # Check boundaries (simple boundary handling)
        if (current_position[0] < 0 or current_position[0] >= environment.shape[1] or
            current_position[1] < 0 or current_position[1] >= environment.shape[0]):
            if verbose:
                print(f"⚠️ Agent reached boundary at step {step+1}")
            break
    
    total_time = time.time() - start_time
    
    # Convert lists to numpy arrays for efficient analysis
    results['positions'] = np.array(results['positions'])
    results['orientations'] = np.array(results['orientations'])
    results['speeds'] = np.array(results['speeds'])
    results['odor_values'] = np.array(results['odor_values'])
    results['timestamps'] = np.array(results['timestamps'])
    
    # Calculate summary statistics
    total_distance = np.sum([details['distance_moved'] for details in results['step_details']])
    max_odor = np.max(results['odor_values'])
    mean_odor = np.mean(results['odor_values'])
    final_position = results['positions'][-1]
    
    results['summary'] = {
        'total_steps': len(results['step_details']),
        'total_time': total_time,
        'avg_step_time': total_time / len(results['step_details']) if results['step_details'] else 0,
        'total_distance': total_distance,
        'max_odor_encountered': max_odor,
        'mean_odor_encountered': mean_odor,
        'final_position': final_position,
        'distance_from_start': np.linalg.norm(final_position - initial_position)
    }
    
    if verbose:
        print(f"\n✅ Simulation complete!")
        print(f"📊 Summary Statistics:")
        print(f"  Total steps executed: {results['summary']['total_steps']}")
        print(f"  Execution time: {results['summary']['total_time']:.3f}s")
        print(f"  Average step time: {results['summary']['avg_step_time']*1000:.2f}ms")
        print(f"  Total distance traveled: {results['summary']['total_distance']:.2f} units")
        print(f"  Maximum odor encountered: {results['summary']['max_odor_encountered']:.4f}")
        print(f"  Mean odor encountered: {results['summary']['mean_odor_encountered']:.4f}")
        print(f"  Final position: ({final_position[0]:.2f}, {final_position[1]:.2f})")
        print(f"  Distance from start: {results['summary']['distance_from_start']:.2f} units")
    
    return results

print("📋 Simulation runner function defined and ready")

## 8. Real-Time Animation Setup

Now let's create our real-time visualization system that will animate the agent's movement through the odor field.

In [ ]:
def create_animation_from_results(results: Dict[str, Any], 
                                 environment: np.ndarray,
                                 config: Dict[str, Any]) -> tuple:
    """
    Create matplotlib animation from simulation results.
    
    This function demonstrates how to create smooth animations from discrete
    simulation data, showcasing the agent's path through the environment.
    
    Args:
        results: Simulation results dictionary
        environment: Environment array for background
        config: Configuration for visualization settings
        
    Returns:
        Tuple of (figure, animation) objects
    """
    print("🎬 Creating animation from simulation results...")
    
    # Extract visualization configuration
    viz_config = config.get('visualization', {})
    figsize = viz_config.get('figsize', [10, 8])
    fps = viz_config.get('fps', 30)
    theme = viz_config.get('theme', 'scientific')
    
    # Create visualization instance
    viz = SimulationVisualization(
        figsize=tuple(figsize),
        fps=fps,
        theme=theme,
        headless=False  # Interactive mode for notebooks
    )
    
    # Set up environment
    viz.setup_environment(environment)
    
    # Prepare animation data
    positions = results['positions']
    orientations = results['orientations']
    odor_values = results['odor_values']
    num_frames = len(positions)
    
    print(f"  Animation frames: {num_frames}")
    print(f"  Frame rate: {fps} FPS")
    print(f"  Duration: {num_frames/fps:.1f} seconds")
    
    # Define frame update function
    def update_frame(frame_idx: int) -> tuple:
        """Generate data for animation frame."""
        if frame_idx < len(positions):
            position = tuple(positions[frame_idx])
            orientation = orientations[frame_idx]
            odor_value = odor_values[frame_idx]
            return (position, orientation, odor_value)
        else:
            # Hold final frame
            position = tuple(positions[-1])
            orientation = orientations[-1]
            odor_value = odor_values[-1]
            return (position, orientation, odor_value)
    
    # Create animation
    anim = viz.create_animation(
        update_func=update_frame,
        frames=num_frames,
        interval=int(1000/fps),  # milliseconds between frames
        blit=True,
        repeat=True
    )
    
    print("✓ Animation created successfully")
    return viz.fig, anim, viz

def run_complete_simulation():
    """
    Run complete simulation workflow with animation.
    
    This function ties together all the components we've built:
    configuration, navigation, simulation, and visualization.
    """
    if WIDGETS_AVAILABLE and output_area:
        with output_area:
            clear_output()
            
    print("🎯 Running complete simulation workflow...")
    
    # Update configuration from widgets
    current_config = update_config_from_widgets()
    
    # Set seed for reproducibility
    if WIDGETS_AVAILABLE:
        current_seed = seed_widget.value
    else:
        current_seed = EXPERIMENT_SEED
        
    with seed_context(current_seed, f"{EXPERIMENT_ID}_run"):
        print(f"🌱 Using seed: {current_seed}")
        
        # Create navigator with current configuration
        current_navigator = create_navigator_from_config(current_config)
        
        # Run simulation
        print("\n" + "="*60)
        sim_results = run_simulation_step_by_step(
            navigator=current_navigator,
            environment=odor_field,
            duration=current_config['simulation']['duration'],
            timestep=current_config['simulation']['timestep'],
            verbose=True
        )
        print("="*60)
        
        # Create and display animation
        print("\n🎬 Generating animation...")
        fig, anim, viz = create_animation_from_results(sim_results, odor_field, current_config)
        
        # Display the animation
        plt.show()
        
        return sim_results, anim, viz

# Connect run button to simulation function if widgets are available
if WIDGETS_AVAILABLE:
    def run_button_clicked(button):
        """Handle run button click."""
        try:
            with output_area:
                results, anim, viz = run_complete_simulation()
                # Store results for later use
                globals()['latest_results'] = results
                globals()['latest_animation'] = anim
                globals()['latest_viz'] = viz
        except Exception as e:
            with output_area:
                print(f"❌ Error during simulation: {str(e)}")
                import traceback
                traceback.print_exc()
    
    run_button.on_click(run_button_clicked)
    print("✓ Interactive simulation controls ready")
else:
    print("📝 Use run_complete_simulation() function to execute simulation")

print("🎞️ Animation system ready")

## 9. Execute the Simulation

Now let's run our simulation! If you have interactive widgets enabled, use the controls above. Otherwise, we'll run with the default configuration.

In [ ]:
# Run the simulation with current configuration
if not WIDGETS_AVAILABLE:
    print("🚀 Running simulation with default configuration...")
    results, anim, viz = run_complete_simulation()
    
    # Store results for analysis
    latest_results = results
    latest_animation = anim
    latest_viz = viz
    
    print("\n✨ Simulation complete! Animation should appear above.")
else:
    print("🎛️ Use the 'Run Simulation' button in the interactive controls above to execute the simulation.")
    print("📋 You can adjust parameters in real-time and see their effects immediately.")
    print("\n💡 Try modifying:")
    print("  • Starting position to see different trajectories")
    print("  • Orientation to change initial direction")
    print("  • Speed to affect movement rate")
    print("  • Duration to run longer or shorter simulations")
    print("  • Random seed to explore stochastic variations")

## 10. Trajectory Analysis and Static Visualization

Let's analyze the simulation results and create publication-quality static visualizations of the agent's trajectory.

In [ ]:
def analyze_trajectory_results(results: Dict[str, Any], 
                              environment: np.ndarray) -> Dict[str, Any]:
    """
    Perform comprehensive analysis of simulation trajectory.
    
    This function demonstrates various analytical techniques for understanding
    agent behavior and navigation performance.
    """
    print("📊 Analyzing trajectory results...")
    
    positions = results['positions']
    orientations = results['orientations']
    odor_values = results['odor_values']
    timestamps = results['timestamps']
    
    # Calculate derived metrics
    velocities = np.diff(positions, axis=0) / np.diff(timestamps)[:, np.newaxis]
    speeds = np.linalg.norm(velocities, axis=1)
    
    # Angular analysis
    angular_changes = np.diff(orientations)
    # Handle angle wrapping
    angular_changes = np.where(angular_changes > 180, angular_changes - 360, angular_changes)
    angular_changes = np.where(angular_changes < -180, angular_changes + 360, angular_changes)
    
    # Spatial analysis
    start_pos = positions[0]
    end_pos = positions[-1]
    displacement = np.linalg.norm(end_pos - start_pos)
    path_length = np.sum(np.linalg.norm(np.diff(positions, axis=0), axis=1))
    tortuosity = path_length / displacement if displacement > 0 else float('inf')
    
    # Odor gradient analysis
    odor_gradient = np.diff(odor_values)
    odor_improvements = np.sum(odor_gradient > 0)
    odor_degradations = np.sum(odor_gradient < 0)
    
    # Environmental coverage
    unique_positions = np.unique(np.round(positions), axis=0)
    env_size = environment.shape[0] * environment.shape[1]
    coverage_ratio = len(unique_positions) / env_size
    
    analysis = {
        'basic_metrics': {
            'total_steps': len(positions) - 1,
            'total_time': timestamps[-1] - timestamps[0],
            'start_position': start_pos,
            'end_position': end_pos,
            'displacement': displacement,
            'path_length': path_length,
            'tortuosity': tortuosity
        },
        'kinematic_metrics': {
            'mean_speed': np.mean(speeds),
            'max_speed': np.max(speeds),
            'speed_std': np.std(speeds),
            'mean_angular_change': np.mean(np.abs(angular_changes)),
            'max_angular_change': np.max(np.abs(angular_changes)),
            'angular_change_std': np.std(angular_changes)
        },
        'odor_metrics': {
            'initial_odor': odor_values[0],
            'final_odor': odor_values[-1],
            'max_odor': np.max(odor_values),
            'mean_odor': np.mean(odor_values),
            'odor_std': np.std(odor_values),
            'odor_improvements': odor_improvements,
            'odor_degradations': odor_degradations,
            'net_odor_change': odor_values[-1] - odor_values[0]
        },
        'exploration_metrics': {
            'unique_positions': len(unique_positions),
            'coverage_ratio': coverage_ratio,
            'bounding_box_area': (
                (np.max(positions[:, 0]) - np.min(positions[:, 0])) *
                (np.max(positions[:, 1]) - np.min(positions[:, 1]))
            )
        }
    }
    
    return analysis

# Run analysis if we have results
if 'latest_results' in globals():
    print("\n📈 Analyzing simulation results...")
    trajectory_analysis = analyze_trajectory_results(latest_results, odor_field)
    
    # Display analysis results
    print("\n🔍 Trajectory Analysis Results:")
    print("\n📏 Basic Metrics:")
    for key, value in trajectory_analysis['basic_metrics'].items():
        if isinstance(value, np.ndarray):
            print(f"  {key}: ({value[0]:.2f}, {value[1]:.2f})")
        else:
            print(f"  {key}: {value:.3f}")
    
    print("\n🏃 Kinematic Metrics:")
    for key, value in trajectory_analysis['kinematic_metrics'].items():
        print(f"  {key}: {value:.3f}")
    
    print("\n👃 Odor Metrics:")
    for key, value in trajectory_analysis['odor_metrics'].items():
        print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")
    
    print("\n🗺️ Exploration Metrics:")
    for key, value in trajectory_analysis['exploration_metrics'].items():
        print(f"  {key}: {value:.3f}" if isinstance(value, float) else f"  {key}: {value}")
    
else:
    print("⚠️ No simulation results available yet. Run a simulation first.")

### 10.1 Create Publication-Quality Static Plots

In [ ]:
# Import visualization function
from {{cookiecutter.project_slug}}.utils.visualization import visualize_trajectory

if 'latest_results' in globals():
    print("🎨 Creating publication-quality trajectory visualization...")
    
    # Extract data for visualization
    positions = latest_results['positions']
    orientations = latest_results['orientations']
    
    # Create comprehensive trajectory plot
    fig = visualize_trajectory(
        positions=positions,
        orientations=orientations,
        plume_frames=odor_field,
        show_plot=True,
        title="Agent Trajectory in Gaussian Odor Field",
        figsize=(14, 10),
        dpi=150,
        theme='scientific',
        start_markers=True,
        end_markers=True,
        orientation_arrows=True,
        trajectory_alpha=0.8,
        grid=True,
        colorbar=True,
        batch_mode=True  # Return figure for further customization
    )
    
    if fig:
        # Add custom annotations
        ax = fig.gca()
        
        # Mark odor sources
        for i, source in enumerate(odor_sources):
            ax.scatter(source['x'], source['y'], c='red', s=150, marker='*', 
                      edgecolors='white', linewidth=2, alpha=0.9, zorder=20)
            ax.annotate(f'Source {i+1}', 
                       xy=(source['x'], source['y']), 
                       xytext=(source['x']+3, source['y']+3),
                       fontsize=10, fontweight='bold', color='darkred',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
        
        # Add trajectory statistics
        if 'trajectory_analysis' in globals():
            stats_text = (
                f"Path Length: {trajectory_analysis['basic_metrics']['path_length']:.1f}\n"
                f"Displacement: {trajectory_analysis['basic_metrics']['displacement']:.1f}\n"
                f"Tortuosity: {trajectory_analysis['basic_metrics']['tortuosity']:.2f}\n"
                f"Max Odor: {trajectory_analysis['odor_metrics']['max_odor']:.3f}"
            )
            ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                   verticalalignment='top', fontsize=9,
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        print("✓ Static trajectory visualization created")
    
else:
    print("⚠️ No simulation results available for visualization. Run a simulation first.")

### 10.2 Time Series Analysis

In [ ]:
if 'latest_results' in globals():
    print("📊 Creating time series analysis plots...")
    
    # Extract time series data
    timestamps = latest_results['timestamps']
    positions = latest_results['positions']
    orientations = latest_results['orientations']
    speeds = latest_results['speeds']
    odor_values = latest_results['odor_values']
    
    # Create comprehensive time series plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Agent Navigation Time Series Analysis', fontsize=16, fontweight='bold')
    
    # Position over time
    axes[0, 0].plot(timestamps, positions[:, 0], 'b-', label='X Position', linewidth=2)
    axes[0, 0].plot(timestamps, positions[:, 1], 'r-', label='Y Position', linewidth=2)
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Position')
    axes[0, 0].set_title('Position vs Time')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Orientation over time
    axes[0, 1].plot(timestamps, orientations, 'g-', linewidth=2)
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Orientation (degrees)')
    axes[0, 1].set_title('Orientation vs Time')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Speed over time
    axes[1, 0].plot(timestamps, speeds, 'm-', linewidth=2)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Speed (units/s)')
    axes[1, 0].set_title('Speed vs Time')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Odor concentration over time
    axes[1, 1].plot(timestamps, odor_values, 'orange', linewidth=2)
    axes[1, 1].fill_between(timestamps, odor_values, alpha=0.3, color='orange')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Odor Concentration')
    axes[1, 1].set_title('Odor Encounter vs Time')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Highlight maximum odor point
    max_odor_idx = np.argmax(odor_values)
    axes[1, 1].scatter(timestamps[max_odor_idx], odor_values[max_odor_idx], 
                      c='red', s=100, marker='*', zorder=10,
                      label=f'Max: {odor_values[max_odor_idx]:.3f}')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("✓ Time series analysis plots created")
    
else:
    print("⚠️ No simulation results available for time series analysis. Run a simulation first.")

## 11. Save Results and Export Options

Finally, let's provide options to save our results and animations for further analysis or presentation.

In [ ]:
def save_simulation_results(results: Dict[str, Any], 
                           config: Dict[str, Any],
                           output_dir: str = "output") -> Dict[str, str]:
    """
    Save simulation results and configuration for reproducibility.
    
    This function demonstrates best practices for research data management
    and reproducible research workflows.
    """
    print(f"💾 Saving simulation results to '{output_dir}'...")
    
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # Generate timestamp for unique filenames
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    saved_files = {}
    
    try:
        # Save trajectory data as NumPy arrays
        trajectory_file = output_path / f"trajectory_{timestamp}.npz"
        np.savez(
            trajectory_file,
            positions=results['positions'],
            orientations=results['orientations'],
            speeds=results['speeds'],
            odor_values=results['odor_values'],
            timestamps=results['timestamps']
        )
        saved_files['trajectory'] = str(trajectory_file)
        print(f"  ✓ Trajectory data: {trajectory_file}")
        
        # Save configuration as JSON
        config_file = output_path / f"config_{timestamp}.json"
        import json
        
        # Convert numpy arrays in config to lists for JSON serialization
        config_serializable = {}
        for key, value in config.items():
            if isinstance(value, dict):
                config_serializable[key] = {}
                for subkey, subvalue in value.items():
                    if isinstance(subvalue, np.ndarray):
                        config_serializable[key][subkey] = subvalue.tolist()
                    elif isinstance(subvalue, (list, tuple)) and len(subvalue) > 0 and isinstance(subvalue[0], dict):
                        config_serializable[key][subkey] = subvalue
                    else:
                        config_serializable[key][subkey] = subvalue
            else:
                config_serializable[key] = value
        
        with open(config_file, 'w') as f:
            json.dump(config_serializable, f, indent=2)
        saved_files['config'] = str(config_file)
        print(f"  ✓ Configuration: {config_file}")
        
        # Save summary statistics
        if 'summary' in results:
            summary_file = output_path / f"summary_{timestamp}.json"
            summary_serializable = {}
            for key, value in results['summary'].items():
                if isinstance(value, np.ndarray):
                    summary_serializable[key] = value.tolist()
                else:
                    summary_serializable[key] = value
            
            with open(summary_file, 'w') as f:
                json.dump(summary_serializable, f, indent=2)
            saved_files['summary'] = str(summary_file)
            print(f"  ✓ Summary statistics: {summary_file}")
        
        # Save reproducibility information
        repro_file = output_path / f"reproducibility_{timestamp}.json"
        repro_info = seed_manager.get_reproducibility_info()
        with open(repro_file, 'w') as f:
            json.dump(repro_info, f, indent=2, default=str)
        saved_files['reproducibility'] = str(repro_file)
        print(f"  ✓ Reproducibility info: {repro_file}")
        
        print(f"\n✅ All files saved successfully to: {output_path.absolute()}")
        
    except Exception as e:
        print(f"❌ Error saving files: {str(e)}")
        import traceback
        traceback.print_exc()
    
    return saved_files

def save_animation_video(animation_obj, output_dir: str = "output", filename: str = None) -> str:
    """Save animation as MP4 video file."""
    from datetime import datetime
    
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"simulation_animation_{timestamp}.mp4"
    
    video_file = output_path / filename
    
    print(f"🎬 Saving animation to: {video_file}")
    
    try:
        if 'latest_viz' in globals():
            latest_viz.save_animation(
                str(video_file),
                fps=30,
                quality='high'
            )
            print(f"✅ Animation saved successfully")
            return str(video_file)
        else:
            print("⚠️ No visualization object available")
            return None
            
    except Exception as e:
        print(f"❌ Error saving animation: {str(e)}")
        return None

# Provide save options if results are available
if 'latest_results' in globals():
    print("💾 Save Options Available:")
    print("\n📁 To save results, run:")
    print("   saved_files = save_simulation_results(latest_results, config)")
    print("\n🎬 To save animation video, run:")
    print("   video_file = save_animation_video(latest_animation)")
    print("\n📊 Files will be saved to 'output/' directory with timestamp")
    
    # Auto-save option (uncomment to enable)
    # print("\n🔄 Auto-saving results...")
    # saved_files = save_simulation_results(latest_results, config)
    # video_file = save_animation_video(latest_animation)
    
else:
    print("ℹ️ Save options will become available after running a simulation.")

## 12. Summary and Next Steps

### What We've Accomplished

Congratulations! You've successfully completed the Simple Agent Movement demonstration. Here's what we covered:

#### 🎯 **Key Learning Outcomes**

1. **Navigation Fundamentals**: You learned how agents are represented with position, orientation, speed, and angular velocity parameters

2. **Environment Interaction**: You saw how agents sample odor concentrations from Gaussian-distributed environmental fields

3. **Configuration Management**: You experienced dynamic parameter assembly using Hydra's Compose API for flexible experimentation

4. **Reproducible Research**: You applied seed management techniques ensuring consistent experimental results across runs

5. **Real-time Visualization**: You created interactive animations showing agent movement through odor landscapes

6. **Data Analysis**: You performed comprehensive trajectory analysis with publication-quality visualizations

#### 🛠️ **Technical Skills Gained**

- **Library Usage**: Importing and using the refactored odor plume navigation library structure
- **Factory Patterns**: Creating navigators through configuration-driven factory methods
- **Animation Techniques**: Building real-time matplotlib animations with custom update functions
- **Parameter Exploration**: Using interactive widgets for real-time parameter modification
- **Data Export**: Saving simulation results and animations for further analysis

#### 📊 **Analysis Capabilities**

- **Kinematic Analysis**: Speed, acceleration, and angular velocity tracking
- **Spatial Metrics**: Path length, displacement, tortuosity, and coverage analysis
- **Environmental Interaction**: Odor encounter patterns and gradient-following behavior
- **Time Series Analysis**: Evolution of agent state over simulation duration

---

### 🚀 **Next Steps and Advanced Topics**

Now that you understand the basics, consider exploring these advanced topics:

#### 📚 **Related Notebooks**

1. **`02_gradient_following_demo.ipynb`**: Learn odor gradient-following algorithms
2. **`03_multi_agent_demo.ipynb`**: Explore multi-agent swarm navigation
3. **`04_custom_sensors_demo.ipynb`**: Implement custom sensor configurations
4. **`05_advanced_visualization.ipynb`**: Master advanced plotting and animation techniques

#### 🔬 **Research Extensions**

- **Algorithm Development**: Implement custom navigation strategies using the NavigatorProtocol
- **Sensor Design**: Explore different sensor layouts and sampling strategies
- **Environment Complexity**: Work with real video plume data and turbulent flows
- **Performance Optimization**: Scale simulations to hundreds of agents

#### 🏗️ **Integration Patterns**

- **Kedro Pipelines**: Integrate with data science workflow management
- **MLflow Tracking**: Add experiment tracking and model versioning
- **Cluster Computing**: Scale to high-performance computing environments
- **Real-time Systems**: Connect to live sensor feeds and robotic systems

---

### 📖 **Key Code Patterns to Remember**

```python
# Configuration-driven initialization
navigator = NavigatorFactory.single_agent(position=(x, y), orientation=θ)

# Reproducible experimentation
with seed_context(seed=42) as sm:
    results = run_simulation()

# Real-time visualization
viz = SimulationVisualization.from_config(cfg.visualization)
animation = viz.create_animation(update_func, frames=N)

# Comprehensive analysis
analysis = analyze_trajectory_results(results, environment)
```

### 🤝 **Community and Support**

- **Documentation**: Comprehensive API documentation and tutorials
- **Examples**: Additional example notebooks and scripts
- **Research Community**: Connect with other researchers using the library
- **Contributions**: Contribute your own algorithms and improvements

---

**Happy researching! 🌟**

*You now have the foundation to explore the fascinating world of odor plume navigation and develop your own navigation algorithms.*